<a href="https://colab.research.google.com/github/senecamiller/ceudemo/blob/main/SM_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!!! HTML SCRAPING / CREATING A DATAFRAME !!!

In [1]:
!pip install scrapethat

In [2]:
from scrapethat import *

In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [4]:
t = read_html('https://www.plane4you.eu/en/offers/single-engine')

In [5]:
def get_links(url):
    t = read_html(url)
    flight_boxes = t.select('.featured-item')
    links = [plane.select_one('a')['href'] for plane in flight_boxes]
    return(links)

In [7]:
def process_one_plane(link):
    t = read_html(link)
    info_rows = t.select('tr')
    flight_data = {}
    flight_data['price']= t.select_one('.up-content').select_one('span').text
    for one_row in info_rows:
        try:
            key = one_row.select_one('.spec-term').text
            value = one_row.select_one('.text-blue').text
            flight_data[key]=value
        except:
            None
    flight_data['url']=link
    return(flight_data)

In [6]:
main_pages=['https://www.plane4you.eu/en/offers/single-engine', 'https://www.plane4you.eu/en/offers/multi-engine', 'https://www.plane4you.eu/en/offers/helicopters', 'https://www.plane4you.eu/en/offers/jet-aircraft', 'https://www.plane4you.eu/en/offers/turboprops', 'https://www.plane4you.eu/en/offers/vintage-project-static', 'https://www.plane4you.eu/en/offers/ultralight-aircraft']
all_links = [links for one_main_page in main_pages for links in get_links(one_main_page)]

In [8]:
final_df = pd.DataFrame(list(map(process_one_plane, tqdm(all_links))))
final_df

100%|██████████| 115/115 [02:37<00:00,  1.37s/it]


,price,Location,Year,TTAF,Serial number,Registration,Reason of sale,Owned by private person,Availability,Type of flights,...,Additional factory equipment,Engine equipment,Avionics system,Guardianavionics,Lights,VFR Night,Instruments,Engine model + HP,Additional option,Recovery system
0,COMING SOON,Poland,2014,1037h,24-0345,SP-WKX (EASA-Poland),Other aviation project in progress,Owned by company - selling based on invoice,Immediately,Private Flights,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,460 000 $,Poland,1981,6457h,172S10770,SP-THL (EASA-Poland),Other aviation project in progress,Owned by company - selling based on invoice,Immediately,Private Flights,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,COMING SOON,Poland,2015,5842h,F15001127,SP-KIZ (EASA-Poland),Other aviation project in progress,NaN,Immediately,Flight training,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,COMING SOON,Poland,1970,8971h,T20608855,SP-HAV (EASA - Poland),Other aviation project in progress,NaN,Immediately,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,419 000 â¬,Spain,1963,2633h,3630,SP-SZP (EASA - Poland),Other aviation project in progress,NaN,Immediately,Private flights,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,145 000 â¬,Portugal,1978,10764h,20 SI 198,OK-AUA32 (UL category - Czech Republic),Other aviation project in progress,NaN,Immediately,Private flights,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,130 000â¬,Slovenia,2008,1050h,1050 SWN 100,"SP-SKPL (Poland, Ultralight)",Other aviation project in progress,NaN,Immediately,Private flights,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,RECENTLY SOLD,Poland,2015,1510h,18 SI 162,SP-SAGH (UL category - Poland - EASA),Other aviation project in progress,NaN,Immediately,Private flights,...,Hook for towing gliders or banners.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,REDUCED PRICE 108 000â¬,Poland,1997,3986h,CH-255C,"K6A - Poland, Amateur category - built by th...",Other aviation project in progress,NaN,Immediately,Private VFR Flights,...,NaN,Sensors: RPM + Carb heat (x2)+ Fuel Press (x2...,"Kanardia NESIS III screen 8.4"" + OAT + CO dete...",smartPanel Mount for Apple iPad Mini with cool...,AveoEnginnering Ultra DayLite â nav/pos/stro...,"Night vfr kit (dimmer, cockpit and instrument ...",Kanardia Artificial Horizon HORIS Master (sepa...,NaN,NaN,NaN


In [9]:
final_df.to_csv('plane_data.csv',   index=False)

In [10]:
from google.colab import files
files.download('plane_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
if os.path.exists('plane_data.csv'):
    final_df = pd.read_csv('plane_data.csv')
else:
    final_df = pd.DataFrame(list(map(process_one_plane, tqdm(all_links))))

!!! DATA CLEANING !!!

In [12]:
# Keep the original DataFrame intact and create a cleaned version (copy)
final_df_copy = final_df.copy()

# Make Column Names Lowercase (easier to explore)
final_df_copy.columns = final_df_copy.columns.str.lower()

# Drop all columns that contain more than 100 null variables
final_df_cleaned = final_df_copy.loc[:, final_df_copy.isna().sum() <= 100]

In [13]:
# Used final_df_cleaned['column-name'].unique() and final_df_cleaned['column-name'].nunique() to learn about the data
# For more info I used some_list = list(final_df_cleaned['column-name']) then some_list.count('value-I-wanted-counted')

# In 'price' category, change USD -> $ (for consistency, both are present) and â\x82¬ -> € (website confirms the two are equal)
final_df_cleaned.loc[:, 'price'] = final_df_cleaned['price'].replace(r'[^\d.,]', '', regex=True)
final_df_cleaned.loc[:, 'price'] = final_df_cleaned['price'].str.replace('USD', '$', regex=False).str.replace('â\x82¬', '€', regex=False)
final_df_cleaned.loc[:, 'price'] = pd.to_numeric(final_df_cleaned['price'], errors='coerce')

# Convert similar column values in 'taxes' to same
# many variations of "Price + VAT (VAT rate depends on Buyer's status)" -- ex: case insensitive
final_df_cleaned.loc[:, 'taxes'] = final_df_cleaned['taxes'].apply(
    lambda x: 'Price + VAT (VAT rate depends on Buyer\'s status)'
    if isinstance(x, str) and 'price + vat' in x.lower() and 'vat rate' in x.lower() else x
)

In [14]:
# Clean 'visual condition' column to remove duplicate values (8//10 vs 8/10 and 9.5/10 vs 9,5/10) and remove /10 to make visualizations easier
final_df_cleaned.loc[:, 'visual condition'] = final_df_cleaned['visual condition'].apply(
    lambda x: '10' if isinstance(x, str) and '10/10' in x
    else '9' if isinstance(x, str) and '9/10' in x
    else '9.5' if isinstance(x, str) and '9,5' in x
    else '8' if isinstance(x, str) and '8//10' in x
    else x.replace('/10', '') if isinstance (x,str) and '/10' in x
    else x
)

# Clean 'owner' into private owner vs. a company selling the plane
final_df_cleaned.loc[:, 'owner'] = final_df_cleaned['owner'].apply(
    lambda x: 'Private Owner' if isinstance(x, str) and ('owner' in x.lower() or 'person' in x.lower() or 'vat paid' in x.lower())
    else 'Private Company' if isinstance(x, str) and ('company' in x.lower() or 'local aeroclub' in x.lower())
    else x
)

# Remove row with year = '.' and ttaf = '.' (1 row each)
final_df_cleaned = final_df_cleaned[final_df_cleaned['price'] != '.']
final_df_cleaned = final_df_cleaned[final_df_cleaned['ttaf'] != '.']

In [15]:
final_df_cleaned.to_csv('clean_plane_data.csv',   index=False)

In [16]:
from google.colab import files
files.download('clean_plane_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
if os.path.exists('clean_plane_data.csv'):
    final_df_cleaned = pd.read_csv('clean_plane_data.csv')
else:
    print('There is an error with final_df_cleaned')

!!! VISUALIZATIONS !!!

In [18]:
import plotly
import plotly.express as px
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

--- Average Price by Year Built ---

In [19]:
# strip ['price'] to only numeric values
#final_df_cleaned['price'] = final_df_cleaned['price'].replace(r'[^\d.,]', '', regex=True)

# Ensure ['price'] and ['year'] are numeric
final_df_cleaned['price'] = pd.to_numeric(final_df_cleaned['price'], errors='coerce')
final_df_cleaned['year'] = pd.to_numeric(final_df_cleaned['year'], errors='coerce')

# Define and label year bins
year_bins = [1960, 1970, 1980, 1990, 2000, 2010, 2020, 2030]
year_labels = ["1960s", "1970s", "1980s", "1990s", "2000s", "2010s", "2020s"]

# Create a new column 'year_bin' with the categorized bins
final_df_cleaned['year_bin'] = pd.cut(
    final_df_cleaned['year'],
    bins=year_bins,
    labels=year_labels,
    right=False
)

# Group by the 'year_bin' and calculate the average 'price' for each category
# Round to Whole Number
average_price = final_df_cleaned.groupby('year_bin', as_index=False)['price'].mean().round()
average_price['price'] = average_price['price'].astype(int)
average_price

<ipython-input-19-15d1dbdeb61e>:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_price = final_df_cleaned.groupby('year_bin', as_index=False)['price'].mean().round()


,year_bin,price
0,1960s,294833
1,1970s,331077
2,1980s,399938
3,1990s,100000
4,2000s,152288
5,2010s,486500
6,2020s,767600


In [20]:
# Create and edit line graph
title = "Average Airplane Price By Decade Built"
chart_size = {'height' : 550, 'width' : 1200}

fig = px.line(average_price, x = 'year_bin', y ='price',
             title = title,
             template = 'ggplot2',
             text='price'
            )

fig.update_traces(textposition='top left')

fig.update_layout(chart_size,
                  xaxis_title = "",
                  yaxis_title = "",
                  title_x = 0.5,
                  font_size = 15,
                  plot_bgcolor = 'white',
                  title_subtitle_text = 'This chart shows the average price of planes offered on plane4you.eu based on year build',
                 )

fig.update_yaxes(showticklabels=False,
                 ticks = "")

fig.show()

--- Plane Visual Condition by Owner Type ---

In [21]:
# Removing null data, creating rating bins and labels, then creating a new columns containing these bins and labels
final_df_cleaned['visual condition'] = pd.to_numeric(final_df_cleaned['visual condition'], errors='coerce')
bins = [5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 11]
labels = ['5', '5.5', '6', '6.5', '7', '7.5', '8', '8.5', '9', '9.5', '10']
final_df_cleaned['visual_condition_group'] = pd.cut(final_df_cleaned['visual condition'],
                                                     bins=bins,
                                                     labels=labels,
                                                     right=False)

In [22]:
# Groups data by two labels, counts the rows for each group, and converts it back to a DataFrame
visual_10 = final_df_cleaned.groupby(['owner','visual_condition_group']).size().reset_index(name='count')

owner_totals = visual_10.groupby('owner')['count'].sum().reset_index()

# Creating and editing a grouped bar chart
title = "Visual Plane Condition Sorted by Seller Type"
chart_size = {'height' : 550, 'width' : 1200}

fig = px.bar(visual_10,
             x = 'count',
             y ='owner',
             color='visual_condition_group',
             title = title,
             template = 'ggplot2',
             labels={'visual_condition_group': 'Visual Rating'}
            )

fig.update_layout(chart_size,
                  xaxis_title = "",
                  yaxis_title = "",
                  title_x = 0.5,
                  font_size = 15,
                  plot_bgcolor = 'white',
                  title_subtitle_text = 'This chart shows the condition of the plane on a scale of 10 grouped in either a private owner or private company',
                  barmode='group'
                 )

fig.update_yaxes(ticks = "")

fig.update_xaxes(showticklabels=False,
                 ticks = "")

fig.show()

<ipython-input-22-22566d9cb8ea>:2: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



--- Remaining Engine Hours by Plane Location ---

In [23]:
# Make a copy of ['remaining hours on engine']  and ['location'] columns
rhoe_cleaned = final_df_cleaned['remaining hours on engine'].copy()
location_copy = final_df_cleaned['location'].copy()

# Filter unwanted values and NaNs
unwanted_values = ['On condition', '396h+ 200h extension']
filtered_df = final_df_cleaned[
    ~final_df_cleaned['remaining hours on engine'].isin(unwanted_values)
    ].dropna(subset=['remaining hours on engine', 'location'])

# Remove 'h' from the 'remaining hours on engine' column
filtered_df['remaining hours on engine'] = (
    filtered_df['remaining hours on engine']
    .str.replace(r'h', '', regex=True)
    .pipe(pd.to_numeric, errors='coerce')
    )

# Group by ['location'], find max/min for ['remaining hours on engine']
stats_by_location = filtered_df.groupby('location')['remaining hours on engine'].agg(
    max_hours='max',
    min_hours='min'
).reset_index()

# Sort the resulting dataframe by 'max_hours' in descending order
stats_by_location = stats_by_location.sort_values(by='max_hours', ascending=False)

stats_by_location

,location,max_hours,min_hours
1,Poland,3450,170
0,Czech Republic,2800,38
2,Portugal,1955,160
3,Slovenia,1618,1618
4,Spain,258,5


In [24]:
# Remove the ' h' addition to numerical columns for proper plotting
stats_by_location['max_hours'] = stats_by_location['max_hours'].astype(float)
stats_by_location['min_hours'] = stats_by_location['min_hours'].astype(float)

# Melt the DataFrame for grouped bar plotting
plot_df = stats_by_location.melt(id_vars='location', var_name='Hour Type', value_name='Hours')

# Create chart
title = "Maximum and Minimum Remaining Hours on Engine Based on Plane Location"
chart_size = {'height': 550, 'width': 1200}

fig = px.bar(
    plot_df,
    x='location',
    y='Hours',
    color='Hour Type',
    color_discrete_map={'max_hours':'thistle', 'min_hours':'skyblue'},
    title=title,
    template='ggplot2',
    text='Hours'
)

# Update layout
fig.update_layout(
    chart_size,
    xaxis_title="",
    yaxis_title="",
    title_x=0.5,
    font_size=15,
    legend_title=None,
    legend=dict(x=0.85, y=0.8),
    plot_bgcolor='white',
    title_subtitle_text="'On condition' and NaN values were removed. The data was grouped by plane sale location.",
    barmode='group'
)

fig.update_xaxes(ticks = "")

fig.update_yaxes(showticklabels=False,
                 ticks = "")

# Format text labels to include ' h'
fig.update_traces(texttemplate='%{text:.0f} h',
                  textposition='inside'
                  )

# Show the chart
fig.show()
